### 인공뉴런

$ y = Sigmoid(X * W + b) $

### 활성화 함수(activation function)
인공신경망을 통과해온 값을 최종적으로 어떤 값으로 만들지를 결정.

Sigmoid, ReLU, tanh 함수등이 있음  

---
- Sigmoid  
0과 1에 한없이 가까워짐
- tanh  
1과 -1에 한없이 가까워짐
- ReLU  
0보다 작으면 항상 0, 0보다크면 입력값을 그대로 출력

### Classification
패턴을 파악해 여러 종류로 구분하는 작업

In [23]:
import tensorflow as tf
import numpy as np

[털, 날개] 특징데이터

In [24]:
x_data = np.array(
[[0, 0],[1, 0],[1, 1],[0, 0],[0, 0],[0, 1]])

각 개체가 실제 어떤 종류인지를 나타내는 레이블(분류값) 데이터를 구성  
`one-hot encoding`을 사용하여 구성
- 기타 = [1, 0, 0]
- 포유류 = [0, 1, 0]
- 조류 = [0, 0, 1]

In [25]:
y_data = np.array([
    [1, 0, 0], #[0, 0] 기타
    [0, 1, 0], #[1, 0] 포유류
    [0, 0, 1], #[1, 1] 조류
    [1, 0, 0], #[0, 0] 기타
    [1, 0, 0], #[0, 0] 기타
    [0, 0, 1], #[0, 1] 조류
])

### 신경망 모델 구성

특징(feature) X와 레이블(label) Y와의 관계를 알아내는 신경망 모델

In [26]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

신경망을 결정하는 가중치(Weight)와 편향(bias)변수 설정  
이 값들을 여러 가지로 바꿔가면서 X와 Y의 연관 관계를 학습

- W : [feature수, label수]로 설정
- b : label수로 설정

### 단일 신경망
---
```python
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))
```

### 다층 신경망
---
입력층과 출력층은 각각 특징과 분류 개수로 맞추고,  
중간의 연결 부분은 맞닿은 층의 뉴런 수와 같도록 맞추면 된다.

중간의 연결 부분을 `은닉층(hidden layer)`이라 하며,  
은닉층의 뉴런 수는 hyperparameter이다.
```py
# weight
W1 = [2, 10] -> [특징, 은닉층의 뉴런 수]
W2 = [10, 3] -> [은닉층의 뉴런 수, 분류 수]
# bias
b1 = [10] -> 은닉층의 뉴런 수
b2 = [3] -> 분류 수
```

In [27]:
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

### 단일신경망
---
```py
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)
```

In [28]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

### Softmax
결과 값들을 전체합이 1이 되도록 만들어주는 함수
> [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]

### 단일신경망
---
```py
model = tf.nn.softmax(L)
```

In [29]:
model = tf.add(tf.matmul(L1, W2), b2)

### Cross-Entropy
- one-hot encoding 을 사용하는 대부분의 모델에서 사용
- 예측값과 실제값 사이의 확률분포 차이를 계산한 값

### 단일신경망
---
```py
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))
```

In [30]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

Y(실측값)     model(신경망을 통해 나온 예측값)  
[1 0 0] [0.1 0.7 0.2]

Y * tf.log(model)  
[-1.0 0]  

tf.reduce_sum : 행별로 값을 다 더함  
[-1.0]

tf.reduce_mean : 평균을 구함 = 우리의 손실값인 Cross-Entropy의 값

### reduce_xxx 함수
- 텐서의 차원을 줄여준다.  
- axis parameter로 축소할 차원을 지정

### GradientDescentOptimizer
---
```py
#Basic한 Gradient Descent 로 최적화
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost)
```

In [36]:
optimizer = tf.train.AdamOptimizer(learning_rate= 0.01)
train_op = optimizer.minimize(cost)

In [37]:
#Tensorflow의 Session을 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [38]:
#앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict = {X: x_data, Y: y_data})
    
    #학습 도중 10번에 한 번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict = {X: x_data, Y: y_data}))

10 0.623132
20 0.478298
30 0.374982
40 0.296026
50 0.232076
60 0.179489
70 0.137172
80 0.104261
90 0.0793409
100 0.060889


In [39]:
#tf.argmax : 요소중 가장 큰 값의 index를 찾아주는 함수, one-hot incoding의 결과를 거꾸로 보여줌
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [40]:
is_correct = tf.equal(prediction, target)
print(sess.run(is_correct, feed_dict={X:x_data, Y:y_data}))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

[ True  True  True  True  True  True]
정확도: 100.00
